In [58]:
#| hide
#| eval:false
! [ -e /content ] && pip install -Uqq fastai # upgrade fastai on colab

In [59]:
#| default_exp l2r.data.load

In [60]:
#| export
from fastai.torch_imports import *
from fastai.data.load import DataLoader
from xcube.imports import *

In [61]:
#| hide
from nbdev.showdoc import *

In [62]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# L2R DataLoader

In [64]:
#| export
class L2RDataLoader(DataLoader):
    def __init__(self, *args, **kwargs):
        self.sl, self.lbs_chunks = kwargs.pop('sl', None), kwargs.pop('lbs_chunks', None)
        if self.sl is None: self.sl = 64
        if self.lbs_chunks is None: self.lbs_chunks = 4
        super().__init__(*args, **kwargs)
        
    def randomize(self):
        seed = np.random.default_rng().integers(0, 2**32-1, 1).item()
        self.rng = np.random.default_rng(seed)
    
    def shuffle_fn(self, idxs): return self.rng.permutation(idxs)

    def get_idxs(self):
        if self.n is not None: idxs = range(self.n)
        if self.shuffle: idxs = (idx for idx in self.shuffle_fn(idxs))
        return idxs
    
    def create_batch(self, start_idx):
        return self.dset[start_idx: min(start_idx+self.bs, self.dset.shape[0])]
        # if self.device: to_device(btch, self.device)
        # return btch
        
    def __len__(self):
        return int(np.ceil((np.ceil(self.dataset.shape[1]/self.sl) * self.lbs_chunks)/self.bs))
    
    def before_iter(self):
        # shuffling
        randperm = torch.randint(low=0, high=self.dataset.shape[1], size=(self.dataset.shape[1],))
        self.dataset = self.dataset[:, randperm]
        # self.lbs_chunks = 4
        size_of_dim0 = torch.ceil(self.dataset.new_empty(1).fill_(self.dataset.shape[0]/self.lbs_chunks)).item()
        pad_len_dim0 = int(self.lbs_chunks * np.floor(self.dataset.shape[0]/self.lbs_chunks) + self.lbs_chunks - self.dataset.shape[0])
        self.dataset_pad = F.pad(self.dataset, (0,0,0,0,0,pad_len_dim0), value=-1)

        trn_sqs = list(torch.split(self.dataset_pad, split_size_or_sections=self.sl, dim=1))
        test_eq(len(trn_sqs), np.ceil(self.dataset_pad.shape[1]/self.sl))
        test_eq(trn_sqs[-1].shape, (self.dataset_pad.shape[0], self.dataset_pad.shape[1]%self.sl,4))
        deficit = self.sl - trn_sqs[-1].shape[1]
        if deficit: 
            test_eq(trn_sqs[-1].shape, (self.dataset_pad.shape[0], self.dataset_pad.shape[1]%self.sl,4));
            # trn_sqs[-1] = torch.concat((trn_sqs[-1], self.dataset_pad.new_empty((trn_sqs[-1].shape[0], deficit,3)).fill_(-1)), dim=1)
            trn_sqs[-1] = trn_sqs[-1].repeat_interleave(self.sl//trn_sqs[-1].shape[1], dim=1)
        test_eq(trn_sqs[-1].shape, (self.dataset_pad.shape[0], self.sl,4));
        # self.dset = torch.concat(trn_sqs)
        # self.dset = torch.stack(trn_sqs)
        
        trn_sqs = map(partial(torch.chunk, chunks=self.lbs_chunks), trn_sqs)
        trn_sqs = itertools.chain.from_iterable(trn_sqs)
        self.dset = trn_sqs
        # test_eq(self.dset.shape, (self.dataset_pad.shape[0]*len(trn_sqs), self.sl, 3))
        # test_eq(self.dset.shape, (len(trn_sqs), self.dataset_pad.shape[0], self.sl, 3))
        # print(f"{self.dset.shape=}")
        # yield from (btch for btch in dset.split(self.bs))
    
    def create_batches(self, samps):
            # trn_sqs = list(torch.split(self.dataset, split_size_or_sections=self.sl, dim=1))
            # test_eq(len(trn_sqs), np.ceil(self.dataset.shape[1]/self.sl))
            # test_eq(trn_sqs[-1].shape, (self.dataset.shape[0], self.dataset.shape[1]%self.sl,3))
            # deficit = self.sl - trn_sqs[-1].shape[1]
            # if deficit: 
            #     test_eq(trn_sqs[-1].shape, (self.dataset.shape[0], self.dataset.shape[1]%self.sl,3));
            #     trn_sqs[-1] = torch.concat((trn_sqs[-1], self.dataset.new_empty((trn_sqs[-1].shape[0], deficit,3)).fill_(-1)), dim=1)
            # test_eq(trn_sqs[-1].shape, (self.dataset.shape[0], self.sl,3));
            # # self.dset = torch.concat(trn_sqs)
            # self.dset = torch.stack(trn_sqs)
            # # test_eq(self.dset.shape, (self.dataset.shape[0]*len(trn_sqs), self.sl, 3))
            # test_eq(self.dset.shape, (len(trn_sqs), self.dataset.shape[0], self.sl, 3))
            # print(f"{self.dset.shape=}")
            # # yield from (btch for btch in dset.split(self.bs))
        # chunks = range(0, self.dset.shape[0], self.bs)
        # with ProcessPoolExecutor(self.n_workers) as ex:
        # with Pool(processes=self.num_workers) as pool:
        # yield from pool.imap_unordered(self.create_batch, chunks, 16)
        # yield from map(self.create_batch, chunks)
        # yield from chunked(self.dset, chunk_sz=self.bs)
        yield from (torch.stack(btch) for btch in self.chunkify(self.dset))
        

## Export

In [65]:
#| hide
import nbdev; nbdev.nbdev_export()